In [3]:
import polars as pl
from google.cloud import storage
import io
import numpy as np
import math
from tqdm import tqdm
from pprint import pprint
import datetime
import numpy as np

In [4]:
bucket_name = "perqara-dendrobium"

# Explicitly use service account credentials by specifying the private key file.
storage_client = storage.Client.from_service_account_json('/content/perqara-data-532572ce4996.json')

# Get the bucket and blob objects
bucket = storage_client.get_bucket(bucket_name)

# Download the contents of the blob as a string
source1 = "raw/postgres/csv/consultations/consultations.csv" # gs uri
blob1 = bucket.blob(source1)
str_data1 = io.BytesIO(blob1.download_as_string())

source2 = "raw/postgres/csv/lawyer_ratings/lawyer_ratings.csv" # gs uri
blob2 = bucket.blob(source2)
str_data2 = io.BytesIO(blob2.download_as_string())

source3 = "raw/mongodb/csv/m_chat_messages/m_chat_messages.csv" # gs uri
blob3 = bucket.blob(source3)
str_data3 = io.BytesIO(blob3.download_as_string())

In [5]:
df = pl.read_csv(str_data1, separator='|')
df.head()

id,parent_id,lawyer_id,client_id,skill_id,description,lawyer_attendance,client_attendance,room_key,lawyer_approved,status,created_at,updated_at,deleted_at,lawyer_approved_at,stop_time,paid_at,retries,last_call,is_client_rated,summary_sent_at,matter,legal_basis,analysis,conclusion
i64,str,i64,i64,i64,str,str,str,str,i64,i64,str,str,str,str,str,str,i64,str,str,str,str,str,str,str
12,null,48,44,2,"""haha""",null,null,"""eyJpdiI6IjZ5T0…",1,600,"""2023-05-02 11:…","""2023-08-24 13:…",null,"""2023-05-02 11:…","""2023-05-02 11:…","""2023-05-02 11:…",0,"""2023-05-02 11:…","""t""",null,"""asdasd""","""asdasd""","""asdasd""","""asdad"""
16,null,48,43,2,"""testing""",null,null,"""eyJpdiI6ImYyVE…",1,600,"""2023-04-11 13:…","""2023-08-11 13:…",null,"""2023-04-11 13:…","""2023-04-11 13:…","""2023-04-11 13:…",0,"""2023-04-11 13:…","""t""",null,"""asdasd""","""asdasd""","""asdasd""","""asdasd"""
17,null,48,43,2,"""test""",null,null,"""eyJpdiI6ImRlM0…",1,600,"""2023-04-12 09:…","""2023-04-12 09:…",null,"""2023-04-12 09:…","""2023-04-12 09:…","""2023-04-12 09:…",0,"""2023-04-12 09:…","""f""",null,"""asdads""","""asdasda""","""sdasd""","""asdasd"""
18,null,48,43,2,"""pringgo test""",null,null,"""eyJpdiI6IjJGLz…",1,600,"""2023-04-12 11:…","""2023-08-24 13:…",null,"""2023-04-12 11:…","""2023-04-12 11:…","""2023-04-12 11:…",0,"""2023-04-12 11:…","""t""",null,"""asdasd""","""asdasd""","""asdasd""","""asdasd"""
19,null,48,49,6,"""testing""",null,null,"""eyJpdiI6Ik1jeV…",1,900,"""2023-04-12 12:…","""2023-04-12 12:…",null,"""2023-04-12 12:…",null,"""2023-04-12 12:…",0,"""2023-04-12 12:…","""f""",null,null,null,null,null


In [6]:
df2 = pl.read_csv(str_data2, separator=',')
df2.head()

id,lawyer_id,consultation_id,client_id,rating,description,status,validated_by_id,created_at,updated_at,deleted_at
i64,i64,i64,i64,i64,str,i64,str,str,str,str
1,48,16,43,5,"""asdasdasd sada…",1,null,"""2023-04-11 14:…","""2023-04-11 14:…",null
2,48,18,43,5,"""asdasasd""",1,null,"""2023-04-12 11:…","""2023-04-12 11:…",null
3,48,21,49,5,""" """,1,null,"""2023-04-12 13:…","""2023-04-12 13:…",null
4,48,20,53,5,"""bagus sekali""",1,null,"""2023-04-12 13:…","""2023-04-12 13:…",null
5,48,24,43,5,"""sdadasd""",1,null,"""2023-04-12 23:…","""2023-04-12 23:…",null


In [7]:
df3 = pl.read_csv(str_data3, separator='|')
df3.head()

consultation_id,user_name,sender_id,message,sent_at,delivered_at,read_at,version,object_id,file_url,file_size,file_name,file_extension,notify_type,notify_description
i64,str,str,str,str,i64,str,i64,str,str,f64,str,str,str,str
4265,"""Pringgo Jr, S.…","""LAWYER""","""hellooo""","""2023-10-11 08:…",0,"""2023-10-11 08:…",0,"""6526578a003319…",null,null,null,null,null,null
4265,"""Daniel Test""","""CLIENT""","""kekirim bang""","""2023-10-11 08:…",0,"""2023-10-11 08:…",0,"""65265794003319…",null,null,null,null,null,null
4265,"""Daniel Test""","""CLIENT""","""masuk ga?""","""2023-10-11 08:…",0,"""2023-10-11 08:…",0,"""65265797003319…",null,null,null,null,null,null
4265,"""Pringgo Jr, S.…","""LAWYER""","""samuk""","""2023-10-11 08:…",0,"""2023-10-11 08:…",0,"""6526579e003319…",null,null,null,null,null,null
4265,"""Daniel Test""","""CLIENT""","""pecah""","""2023-10-11 08:…",0,"""2023-10-11 08:…",0,"""652657a2003319…",null,null,null,null,null,null


In [8]:
def remove_perqara_testing_data(df: pl.DataFrame, list_of_ids: list, column_name: str) -> pl.DataFrame:
  df = df.filter(~pl.col(column_name).is_in(list_of_ids))
  print(f"Dataframe has been successfully filtered based on {column_name} column.")
  return df

In [9]:
list_of_excluded_client_ids = [
    25,
    42,
    43,
    44,
    45,
    49,
    54,
    56,
    58,
    62,
    63,
    66,
    85,
    146,
    157,
    295,
    959,
    1609,
    1655,
    2421,
    2526,
    3180,
    4205,
    49,
    56,
    157,
    2995,
    7569,
    5804,
    5717,
    5716,
    1821,
    7749,
    7743,
    54,
    78,
    11746,
]
list_of_excluded_lawyer_ids = [36, 38, 48, 120, 192, 195]
status_code = 600

In [10]:
cleaned_consultations_df = remove_perqara_testing_data(df, list_of_excluded_client_ids, "client_id")
cleaned_consultations_df = remove_perqara_testing_data(cleaned_consultations_df, list_of_excluded_lawyer_ids, "lawyer_id")

cleaned_consultations_df = cleaned_consultations_df.filter(pl.col("status") == status_code)

cleaned_consultations_df = cleaned_consultations_df.with_columns(
    pl.col("created_at").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False),
    pl.col("stop_time").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False),
    pl.col("updated_at").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False),
)

Dataframe has been successfully filtered based on client_id column.
Dataframe has been successfully filtered based on lawyer_id column.


In [11]:
cleaned_consultations_df.glimpse()

Rows: 16080
Columns: 25
$ id                          <i64> 50, 61, 62, 66, 68, 72, 102, 110, 111, 112
$ parent_id                   <str> None, None, None, None, None, None, None, None, None, None
$ lawyer_id                   <i64> 31, 31, 31, 17, 33, 33, 33, 33, 33, 33
$ client_id                   <i64> 72, 100, 101, 79, 155, 160, 162, 165, 165, 187
$ skill_id                    <i64> 3, 3, 6, 5, 3, 5, 2, 5, 5, 5
$ description                 <str> 'Saya ingin menanyakan masalah teman saya, sudah lama pisah dengan istri tapi belum sempat urus akte cerai karena bekerja di provinsi lain, istrinya bersedia mengurus surat cerai asal diberikan sejumlah uang dan teman Sy tdk mampu memberikan uang sebanyak yang diminta, teman saya tidak bisa mengurus surat cerai Krn susah mengambil cuti, cuti pun hanya 6 hari saja, sedangkan perjalanan dari perusahaan tempat bekerja ke bandara memakan waktu kurang lebih 2 hari, belum lagi pengurusan dan perjalanan plng. Mohon solusinya pak !', 'Hallo pak 

In [12]:
# select relevant columns
cleaned_consultations_df = cleaned_consultations_df.select(["id", "created_at", "stop_time", "updated_at"])
merged_m_chat_messages_df = df3.select(
    ["consultation_id", "sender_id", "message", "sent_at", "read_at"]
)

# Filter chat messages based on consultation IDs
df_chat_filtered = merged_m_chat_messages_df.filter(
    merged_m_chat_messages_df["consultation_id"].is_in(cleaned_consultations_df["id"])
)

cleaned_consultations_df = cleaned_consultations_df.with_columns(
    cleaned_consultations_df["id"].alias("consultation_id")
)

# Merge chat messages and consultations data
merged_df = df_chat_filtered.join(
    cleaned_consultations_df, left_on="consultation_id", right_on="consultation_id", how="left", suffix="_y"
)

In [13]:
merged_df.head()

consultation_id,sender_id,message,sent_at,read_at,id,created_at,stop_time,updated_at
i64,str,str,str,str,i64,datetime[μs],datetime[μs],datetime[μs]
4268,"""LAWYER""","""ad yg bisa say…","""2023-10-11 08:…","""2023-10-11 08:…",4268,2023-10-11 15:17:17,2023-10-11 16:24:02,2023-10-12 15:02:30
4269,"""LAWYER""","""ada yg bisa di…","""2023-10-11 08:…","""1970-01-01 00:…",4269,2023-10-11 15:26:21,2023-10-11 16:24:02,2023-10-11 16:59:48
4270,"""CLIENT""","""Assalamualaiku…","""2023-10-11 08:…","""2023-10-11 08:…",4270,2023-10-11 15:26:44,2023-10-11 16:24:02,2023-10-11 17:00:20
4272,"""LAWYER""","""selamat sore ,…","""2023-10-11 08:…","""2023-10-11 08:…",4272,2023-10-11 15:33:09,2023-10-11 16:23:20,2023-10-11 21:01:03
4272,"""LAWYER""","""ada yg bisa ka…","""2023-10-11 08:…","""2023-10-11 08:…",4272,2023-10-11 15:33:09,2023-10-11 16:23:20,2023-10-11 21:01:03


In [14]:
merged_df = merged_df.with_columns(pl.col('sent_at').str.to_datetime())
merged_df = merged_df.with_columns(pl.col('read_at').str.to_datetime())

In [15]:
def calculate_conversation_duration(df_input: pl.DataFrame, delimiter: str) -> pl.DataFrame:
  # get unique consultation_id
  consultation_unique_ids = df_input.select(pl.col("consultation_id")).unique().to_series().to_list()

  col_consul_id = []
  col_client_read_avg = []
  col_lawyer_read_avg = []
  col_last_time = []
  col_lawyer_messages_count = []
  col_client_messages_count = []
  col_system_messages_count = []
  col_last_sender = []

  # START
  for i in tqdm(consultation_unique_ids):
      # get data for each consultation
      df = df_input.filter(pl.col("consultation_id") == i)

      # initial variables
      timestamps = []
      current_sender = None
      prev_sender = None
      prev_time = None

      # get first and last sender
      first_sender = df["sender_id"][0]
      last_sender = df["sender_id"][-1]

      # START
      for row in df.iter_rows(named=True):
          current_sender = row["sender_id"]
          if current_sender != prev_sender:
              timestamps.append(prev_time)
          prev_sender = current_sender
          prev_time = row["sent_at"]

      # append sent_at
      timestamps.append(row["sent_at"])

      # remove first element
      timestamps = timestamps[1:]

      # Check if sent_at and stop_time are not None
      if row["stop_time"] is not None and row["sent_at"] is not None:
          last_time = (row["stop_time"] - row["sent_at"]).total_seconds()
      else:
          last_time = np.nan  # Assign NaN (Not a Number) if list is empty

      # Filter out None values
      timestamps = [ts for ts in timestamps if ts is not None]

      # Convert the timestamps to numpy datetime64 objects
      np_timestamps = np.array(timestamps, dtype="datetime64")

      # Check if datetime_objects is not empty and has more than one element
      if len(np_timestamps) > 1:
          # Calculate time differences in seconds
          time_diff_seconds = np.diff(np_timestamps) / np.timedelta64(1, "s")
      else:
          # Handle cases where the array is empty or has only one element
          time_diff_seconds = np.array([])  # Empty array

      # Calculate average time between messages
      if first_sender == "LAWYER":
          client_to_lawyer = [time_diff_seconds[i] for i in range(len(time_diff_seconds)) if i % 2 == 0]
          lawyer_to_client = [time_diff_seconds[i] for i in range(len(time_diff_seconds)) if i % 2 != 0]
      else:
          lawyer_to_client = [time_diff_seconds[i] for i in range(len(time_diff_seconds)) if i % 2 == 0]
          client_to_lawyer = [time_diff_seconds[i] for i in range(len(time_diff_seconds)) if i % 2 != 0]

      # Calculate mean only if the lists are not empty
      if client_to_lawyer:
          client_read_avg = np.mean(client_to_lawyer)
      else:
          client_read_avg = np.nan  # Assign NaN (Not a Number) if list is empty

      if lawyer_to_client:
          lawyer_read_avg = np.mean(lawyer_to_client)
      else:
          lawyer_read_avg = np.nan  # Assign NaN (Not a Number) if list is empty

      # Append values
      col_consul_id.append(i)
      col_client_read_avg.append(client_read_avg)
      col_lawyer_read_avg.append(lawyer_read_avg)
      col_last_time.append(last_time)
      col_client_messages_count.append(df.filter(pl.col("sender_id") == "CLIENT").shape[0])
      col_lawyer_messages_count.append(df.filter(pl.col("sender_id") == "LAWYER").shape[0])
      col_system_messages_count.append(df.filter(pl.col("sender_id") == "SYSTEM").shape[0])
      col_last_sender.append(last_sender)
  # END

  # Round values to 2 decimal places
  col_client_read_avg_rounded = [
      round(value, 2) if not math.isnan(value) else value for value in col_client_read_avg
  ]
  col_lawyer_read_avg_rounded = [
      round(value, 2) if not math.isnan(value) else value for value in col_lawyer_read_avg
  ]

  # Create a dataframe
  data = {
      "consultation_id": col_consul_id,
      "client_msg_cnt": col_client_messages_count,
      "lawyer_msg_cnt": col_lawyer_messages_count,
      "system_msg_cnt": col_system_messages_count,
      "client_read_avg": col_client_read_avg_rounded,
      "lawyer_read_avg": col_lawyer_read_avg_rounded,
      "last_time": col_last_time,
      "last_sender": col_last_sender,
  }

  df_result = pl.DataFrame(data)
  return df_result
  # Save DataFrame to CSV file
  # df_result.to_csv(filename, sep=delimiter, index=False)

In [16]:
df_result = calculate_conversation_duration(merged_df, "|")
df_result.head()

100%|██████████| 13787/13787 [00:35<00:00, 391.65it/s]


consultation_id,client_msg_cnt,lawyer_msg_cnt,system_msg_cnt,client_read_avg,lawyer_read_avg,last_time,last_sender
i64,i64,i64,i64,f64,f64,f64,str
16513,1,3,0,105.56,232.92,26562.141,"""LAWYER"""
6932,28,22,0,76.1,43.22,25201.492,"""LAWYER"""
20779,5,11,0,112.85,130.85,25204.651,"""CLIENT"""
15735,25,24,0,38.74,37.63,25204.353,"""CLIENT"""
7158,43,30,0,55.49,42.83,25203.364,"""LAWYER"""


In [17]:
# Selected numeric columns
numeric_columns = ["client_read_avg", "lawyer_read_avg", "last_time"]

# Calculate quartiles
quartile1 = df_result.select(numeric_columns).quantile(0.25)
quartile3 = df_result.select(numeric_columns).quantile(0.75)

In [18]:
print(quartile1)
print(quartile3)

shape: (1, 3)
┌─────────────────┬─────────────────┬───────────┐
│ client_read_avg ┆ lawyer_read_avg ┆ last_time │
│ ---             ┆ ---             ┆ ---       │
│ f64             ┆ f64             ┆ f64       │
╞═════════════════╪═════════════════╪═══════════╡
│ 45.06           ┆ 41.9            ┆ 25207.903 │
└─────────────────┴─────────────────┴───────────┘
shape: (1, 3)
┌─────────────────┬─────────────────┬───────────┐
│ client_read_avg ┆ lawyer_read_avg ┆ last_time │
│ ---             ┆ ---             ┆ ---       │
│ f64             ┆ f64             ┆ f64       │
╞═════════════════╪═════════════════╪═══════════╡
│ 160.61          ┆ 157.18          ┆ 25494.43  │
└─────────────────┴─────────────────┴───────────┘


In [19]:
# Calculate interquartile range (IQR)
iqr = quartile3 - quartile1
print(iqr)

shape: (1, 3)
┌─────────────────┬─────────────────┬───────────┐
│ client_read_avg ┆ lawyer_read_avg ┆ last_time │
│ ---             ┆ ---             ┆ ---       │
│ f64             ┆ f64             ┆ f64       │
╞═════════════════╪═════════════════╪═══════════╡
│ 115.55          ┆ 115.28          ┆ 286.527   │
└─────────────────┴─────────────────┴───────────┘


In [20]:
# Extract values from DataFrames as lists
q1_values = quartile1.to_numpy().flatten().tolist()
q3_values = quartile3.to_numpy().flatten().tolist()
iqr_values = iqr.to_numpy().flatten().tolist()

# Create a dictionary for the final result
result_dict = {
    'metrics': ['client_read_avg', 'lawyer_read_avg', 'last_time'],
    'q1': q1_values,
    'q3': q3_values,
    'iqr': iqr_values
}

# Create a new DataFrame for the final result
merged_df = pl.DataFrame(result_dict)

In [21]:
merged_df.head()

metrics,q1,q3,iqr
str,f64,f64,f64
"""client_read_av…",45.06,160.61,115.55
"""lawyer_read_av…",41.9,157.18,115.28
"""last_time""",25207.903,25494.43,286.527
